<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<br><h2>Script 03 | Combining PCA and Clustering</h2>
<br>
Written by Chase Kusterer<br>
<a href="https://github.com/chase-kusterer">GitHub</a> | <a href="https://www.linkedin.com/in/kusterer/">LinkedIn</a>
<br><br><br>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<h2>Part I: Key Concepts</h2><br>

<strong>Principal Component Analysis</strong><br>
Operates on the variance between the x-features.<br><br>

Three situations where PCA is useful:
1. Correlated X-Features (which cannot be used together in many supervised learning models)
2. Dimensionality Reduction (grouping large variable sets into a more manageable number of factors)
3. Latent Trait Exploration (measuring what cannot be measured directly)


<br><br>
<strong>Clustering</strong><br>
Divide the data into groups, more formally known as clusters. One of the following will occur:
* observations will be grouped based on their similarities
* observations will be separated based on their differences

<br>
Reminder: Distance-based techniques require variance scaling.
<br><br><br>
<font color="red"><strong><u>Don't forget!!!</u></strong></font>

1. Don't mix data concepts in the same algorithm (spending behavior, demographics, psychometrics, etc.). 
2. Scale your data.
3. Interpretation is subjective, so spend ample time on this step.

<br>
<strong>Run the following code to import the necessary packages for this analysis.</strong>

In [ ]:
########################################
# importing packages
########################################
import numpy                 as np  # mathematical essentials
import pandas                as pd  # data science essentials
import matplotlib.pyplot     as plt # fundamental data visualization
import seaborn               as sns # enhanced visualizations

# packages for unsupervised learning
from sklearn.preprocessing   import StandardScaler      # standard scaler
from sklearn.decomposition   import PCA                 # pca
from scipy.cluster.hierarchy import dendrogram, linkage # dendrograms
from sklearn.cluster         import KMeans              # k-means clustering

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<strong>Run the following code to load the dataset and set print options.</strong><br>
Note that the values in each food category are revenue, but the currency units have been masked and the data has been treated for skewness.

In [ ]:
########################################
# loading data and setting display options
########################################
# loading data
customers_df = pd.read_excel('./datasets/top_customers.xlsx')


# setting print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)


# checking results
customers_df.head(n = 10)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<strong>User-Defined Functions</strong><br>
Run the following code to load user-defined functions.

In [ ]:
########################################
# scree_plot
########################################
def scree_plot(pca_object, export = False):
    # building a scree plot

    # setting plot size
    fig, ax = plt.subplots(figsize=(10, 8))
    features = range(pca_object.n_components_)


    # developing a scree plot
    plt.plot(features,
             pca_object.explained_variance_ratio_,
             linewidth = 2,
             marker = 'o',
             markersize = 10,
             markeredgecolor = 'black',
             markerfacecolor = 'grey')


    # setting more plot options
    plt.title('Scree Plot')
    plt.xlabel('PCA feature')
    plt.ylabel('Explained Variance')
    plt.xticks(features)

    if export == True:
    
        # exporting the plot
        plt.savefig('./analysis_images/top_customers_correlation_scree_plot.png')
        
    # displaying the plot
    plt.show()


########################################
# unsupervised_scaler
########################################
def scaler(df):
    """
    Standardizes a dataset (mean = 0, variance = 1). Returns a new DataFrame.
    Requires sklearn.preprocessing.StandardScaler()
    
    PARAMETERS
    ----------
    df     | DataFrame to be used for scaling
    """

    # INSTANTIATING a StandardScaler() object
    scaler = StandardScaler(copy = True)


    # FITTING the scaler with the data
    scaler.fit(df)


    # TRANSFORMING our data after fit
    x_scaled = scaler.transform(df)

    
    # converting scaled data into a DataFrame
    new_df = pd.DataFrame(x_scaled)


    # reattaching column names
    new_df.columns = list(df.columns)
    
    return new_df

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h4>b) Drop demographic information and scale the data.</h4>

In [ ]:
# dropping demographic information
_____


# applying the unsupervised_scaler function
purchases_scaled = scaler(df = ____)



# checking pre- and post-scaling variance
_____

In [ ]:
# dropping demographic information
purchase_behavior = customers_df.drop(['Channel', 'Region'],
                                      axis = 1)


# applying the unsupervised_scaler function
purchases_scaled = scaler(df = purchase_behavior)


# checking pre- and post-scaling variance
print(np.var(purchase_behavior), '\n\n')
print(np.var(purchases_scaled))

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h2>Part II: Principal Component Analysis</h2>

Our process here is to:
1. Develop a PCA model with no limit to principal components
2. Analyze the <strong>explained_variance_ratio</strong> and the <strong>scree plot</strong>
3. Decide how many components to RETAIN
4. Build a new model with a limited number of principal components
5. Interpret the results

<h4>a) Develop a PCA object with no limit to principal components and analyze its scree plot.</h4>

In [ ]:
# INSTANTIATING a PCA object with no limit to principal components
pca = _____(_____,
          random_state = 702)


# FITTING and TRANSFORMING the scaled data
customer_pca = _____._____(_____)


# calling the scree_plot function
_____(pca_object = _____)

In [ ]:
# INSTANTIATING a PCA object with no limit to principal components
pca = PCA(n_components = None,
          random_state = 702)


# FITTING and TRANSFORMING the purchases_scaled
customer_pca = pca.fit_transform(purchases_scaled)


# calling the scree_plot function
scree_plot(pca_object = pca)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h4>b) Reduce the number of principal components to a reasonable number based on the scree plot above.</h4><br>
In this example, we will assume three PCs is a reasonable number based on the elbow in the scree plot. Also note that it would have been reasonable to retain enough PCs so that the cumulative explained variance ratio is greater than or equal to 0.80. Note that we do not need to rerun the scree plot after completing this step.

In [ ]:
# INSTANTIATING a new model using the first three principal components
pca_3 = _____(_____,
            _____ = 702)


# FITTING and TRANSFORMING the purchases_scaled
customer_pca_3 = _____._____(_____)

In [ ]:
# INSTANTIATING a new model using the first three principal components
pca_3 = PCA(n_components = 3,
            random_state = 702)


# FITTING and TRANSFORMING the purchases_scaled
customer_pca_3 = pca_3.fit_transform(purchases_scaled)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<strong>OPTIONAL STEP</strong><br>Run the following code to compare the variance of the unlimited PCA model with the variance of the reduced PCA model. We are doing this in this script simply to show that the explained variance for each principal component does not change after dropping smaller PCs.

In [ ]:
####################
### Max PC Model ###
####################
# transposing pca components (pc = MAX)
factor_loadings = pd.DataFrame(np.transpose(pca.components_))


# naming rows as original features
factor_loadings = factor_loadings.set_index(purchases_scaled.columns)


##################
### 3 PC Model ###
##################
# transposing pca components (pc = 3)
factor_loadings_3 = pd.DataFrame(np.transpose(pca_3.components_))


# naming rows as original features
factor_loadings_3 = factor_loadings_3.set_index(purchases_scaled.columns)


# checking the results
print(f"""
MAX Components Factor Loadings
------------------------------
{factor_loadings.round(2)}


3 Components Factor Loadings
------------------------------
{factor_loadings_3.round(2)}
""")

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h4>c) Analyze and name each principal component based on its factor loading.</h4>

In this step, make sure to develop a story behind what each PC name represents. This is an ideal method for bridging the gap between the technical and non-technical people you are working with. Remember, by doing a good job here you are putting analytics at the forefront of strategic decision making, which is a great way to boost your value within an organization.

In [ ]:
# naming each principal component
factor_loadings_3._____ = ['Protein Over Vitamins', # - Vegan, - Veg, - Indian
                           'Sleeptime Bliss',       # - Med, - ME, - Wine
                           'Palate Practicality']   # + Med,  - Wine


# checking the result
_____

In [ ]:
# naming each principal component
factor_loadings_3.columns = ['Protein Over Vitamins', # - Vegan, - Veg, - Indian
                             'Sleeptime Bliss',       # - Med, - ME, - Wine
                             'Palate Practicality']   # + Med,  - Wine


# checking the result
factor_loadings_3.round(decimals = 2)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h4>d) Analyze the factor loadings for each customer in the dataset.</h4>

Do this by identifying groups of customers that have very high or very low factor loadings in any given principal component. A good heuristic is to look for factor loadings that are greater than one standard deviation from the mean in absolute value. Develop a strategy for key groups that you identify.<br><br>

<strong>Don't forget</strong> to look at both the positive and negative loadings.<br>
<strong>Don't forget</strong> to calculate the percentage of your audience effected by each loading when developing your targeting strategy/new ideas.<br>
<strong>Don't forget</strong> to also consider the proportion of revenue generated by each group.

In [ ]:
# analyzing factor strengths per customer
factor_loadings = pca_3.transform(purchases_scaled)


# converting to a DataFrame
factor_loadings_df = _____._____(_____)


# renaming columns
factor_loadings_df.columns = factor_loadings_3.columns


# checking the results
_____

In [ ]:
# analyzing factor strengths per customer
factor_loadings = pca_3.transform(purchases_scaled)


# converting to a DataFrame
factor_loadings_df = pd.DataFrame(factor_loadings)


# renaming columns
factor_loadings_df.columns = factor_loadings_3.columns


# checking the results
factor_loadings_df.head(n = 5)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

Sending PCA factor loadings to Excel for further analysis.

In [ ]:
factor_loadings_df.to_excel('./analysis_results/PCA Factor Loadings.xlsx',
                            index = False)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h2>Part III: Intro to Clustering</h2><br>
We are going to start by building an agglomerative clustering model. We are primarily interested in the <strong>dendrogram</strong> and the <strong>inertia plot</strong>. Our goal is to develop an idea as to how many clusters would be appropriate given our analysis of these tools, and then to apply this number of clusters to a k-Means model. Try to come away with 4-5 different numbers of clusters so that you have more options when applying k-Means. <strong>Before getting started, we need to rescale our data.</strong> The reason is that the variance amongst our features is no longer equal.

In [ ]:
# checking variance amongst clusters
np.var(factor_loadings_df)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h4>a) Complete the code to prepare a scaled version of the factor loadings dataset.</h4>

In [ ]:
# applying the unsupervised_scaler function
pca_rescaled = _____


# checking pre- and post-scaling variance
print(np.var(factor_loadings_df), '\n\n')
print(np.var(pca_rescaled))

In [ ]:
# applying the unsupervised_scaler function
pca_rescaled = scaler(df = factor_loadings_df)


# checking pre- and post-scaling variance
print(np.var(factor_loadings_df), '\n\n')
print(np.var(pca_rescaled))

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h2>Part IV: Agglomerative Clustering</h2><br>
Agglomerative clustering starts with each observation in its own cluster. From here, it links observations  based on distance. There are three primary methods for calculating distance:<br><br>

    ward (default) - groups observations into clusters in a way that minimizes 
    the variance amongst all clusters. Leads to clusters that are relatively
    equal in size

    average - merges clusters that have the smallest average distance
    between all their points

    complete - merges clusters that have the smallest maximum distance
    between their points

<br><br>
<u>Primary Advantage</u><br>
Able to generate a dendrogram to better understand data groupings and help determine the final number of clusters to develop.
<br><br>
<u>Primary Disadvantage</u><br>
Unable to predict on new data.<br><br>

Run the following code to develop a dendrogram. Our goal here is to understand how many clusters to build using k-Means.

In [ ]:
# grouping data based on Ward distance
standard_mergings_ward = linkage(y = pca_rescaled,
                                 method = 'ward',
                                 optimal_ordering = True)


# setting plot size
fig, ax = plt.subplots(figsize=(12, 12))

# developing a dendrogram
dendrogram(Z = standard_mergings_ward,
           leaf_rotation  = 90       ,
           leaf_font_size = 6        )


# rendering the plot
plt.show()

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h4>a) Complete the code to develop a k-Means model with three clusters.</h4>
This is where we test our candidate number of clusters. When we find a clustering that we like, we move forward. For this example, let's assume we converged on a solution of three clusters.<br><br>
<strong>Don't forget</strong> that the appropriate number of clusters does not have to be the same as the number of principal components that were retained.

In [ ]:
# INSTANTIATING a k-Means object with five clusters
customers_k_pca = _____(n_clusters   = 3     ,
                        n_init       = 'auto',
                        random_state = 702   )


# fitting the object to the data
customers_k_pca._____(_____)


# converting the clusters to a DataFrame
customers_kmeans_pca = _____._____({'Cluster': customers_k_pca.labels_})


# checking the results
print(customers_kmeans_pca.iloc[: , 0].value_counts())

In [ ]:
# INSTANTIATING a k-Means object with clusters
customers_k_pca = KMeans(n_clusters   = 3     ,
                         n_init       = 'auto',
                         random_state = 702   )


# fitting the object to the data
customers_k_pca.fit(pca_rescaled)


# converting the clusters to a DataFrame
customers_kmeans_pca = pd.DataFrame({'Cluster': customers_k_pca.labels_})


# checking the results
print(customers_kmeans_pca.iloc[: , 0].value_counts())

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h4>b) Finish the code to display the centroids (mean values) for each cluster.</h4>
When you are finished, interpret their meaning. This is also a place where you may want to (optionally) name your clusters and develop back stories for ideal members of each group.

In [ ]:
# storing cluster centers
centroids_pca = customers_k_pca._____


# converting cluster centers into a DataFrame
centroids_pca_df = pd.DataFrame(_____).round(decimals = 2)


# renaming principal components
centroids_pca_df._____ = _['Protein Over Vitamins',
                           'Sleeptime Bliss',
                           'Palate Practicality']


# checking results (clusters = rows, pc = columns)
centroids_pca_df.round(2)

In [ ]:
# storing cluster centers
centroids_pca = customers_k_pca.cluster_centers_


# converting cluster centers into a DataFrame
centroids_pca_df = pd.DataFrame(centroids_pca).round(decimals = 2)


# renaming principal components
centroids_pca_df.columns = ['Protein Over Vitamins',
                             'Sleeptime Bliss',
                             'Palate Practicality']


# checking results (clusters = rows, pc = columns)
centroids_pca_df

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h4>c) Run the following code to concatenate channel, region, and the PCA components into one DataFrame.</h4>

In [ ]:
# concatenating cluster memberships with principal components
clst_pca_df = pd.concat([customers_kmeans_pca,
                         factor_loadings_df],
                         axis = 1)


# concatenating demographic information with pca-clusters
final_df = pd.concat([customers_df.loc[ : , ['Channel', 'Region']],
                      clst_pca_df.round(decimals = 2)],
                      axis = 1)


# renaming columns
final_df.columns = ['Channel', 'Region', 'Cluster',
                    'Protein Over Vitamins',
                    'Sleeptime Bliss',
                    'Palate Practicality']


# checking the results
print(final_df.head(n = 5))

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

Run the following code to add labels to categorical variables. If you (optionally) named your clusters, make sure to label these as well.

In [ ]:
# renaming channels
channel_names = {1 : 'Online',
                 2 : 'Mobile'}


final_df['Channel'].replace(channel_names, inplace = True)



# renaming regions
region_names = {1 : 'Alameda',
                2 : 'San Francisco',
                3 : 'Contra Costa'}


final_df['Region'].replace(region_names, inplace = True)


# renaming regions
cluster_names = {0 : '1',
                 1 : '2',
                 2 : '3'}


final_df['Cluster'].replace(cluster_names, inplace = True)


# checking results
final_df.head(n = 5)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h2>Part V: Analyze with Demographics</h2><br>
Now that we've completed all of our preparation through machine learning, we can analyze our results with demographics and other data.<br><br>
<strong>Pause before this step</strong> so that you can consider all of the hypotheses and assumptions you have made up to this point. Also consider all of the assumptions your organization is making. For example, if the company is convinced of a particular trend, the following is a good opportunity to validate/negate that information.

In [ ]:
# dynamic string with value counts for each demographic (cluster 1)
print(f"""\
 -----------
| Cluster 1 |
 -----------

Proportion of Observations
--------------------------
{round(len(final_df.loc[ : , "Cluster"][final_df.loc[ : , "Cluster"] == '1']) /
       len(final_df), ndigits = 2)}


Centroids
---------
{centroids_pca_df.loc[ 0 , :].to_string(dtype = False, name = False)}


Channel
-------
{final_df.loc[ : , "Channel"][ final_df.loc[ : , 'Cluster' ] == '1']

         .value_counts(normalize = True)
         .round(decimals = 2)
         .sort_index().to_string(dtype = False, name = False)}
  
  
Region
------
{final_df.loc[ : , "Region"][ final_df.loc[ : , 'Cluster' ] == '1']

         .value_counts(normalize = True)
         .round(decimals = 2)
         .sort_index().to_string(dtype = False, name = False)}
""")

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

In [ ]:
# dynamic string with value counts for each demographic (cluster 2)
print(f"""\
 -----------
| Cluster 2 |
 -----------

Proportion of Observations
--------------------------
{round(len(final_df.loc[ : , "Cluster"][final_df.loc[ : , "Cluster"] == '2']) /
       len(final_df), ndigits = 2)}


Centroids
---------
{centroids_pca_df.loc[ 1 , :].to_string(dtype = False, name = False)}


Channel
-------
{final_df.loc[ : , "Channel"][ final_df.loc[ : , 'Cluster' ] == '2']

         .value_counts(normalize = True)
         .round(decimals = 2)
         .sort_index().to_string(dtype = False, name = False)}
  
  
Region
------
{final_df.loc[ : , "Region"][ final_df.loc[ : , 'Cluster' ] == '2']

         .value_counts(normalize = True)
         .round(decimals = 2)
         .sort_index().to_string(dtype = False, name = False)}
""")

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

In [ ]:
# dynamic string with value counts for each demographic (cluster 3)
print(f"""\
 -----------
| Cluster 3 |
 -----------

Proportion of Observations
--------------------------
{round(len(final_df.loc[ : , "Cluster"][final_df.loc[ : , "Cluster"] == '3']) /
       len(final_df), ndigits = 2)}


Centroids
---------
{centroids_pca_df.loc[ 2 , :].to_string(dtype = False, name = False)}


Channel
-------
{final_df.loc[ : , "Channel"][ final_df.loc[ : , 'Cluster' ] == '3']

         .value_counts(normalize = True)
         .round(decimals = 2)
         .sort_index().to_string(dtype = False, name = False)}
  
  
Region
------
{final_df.loc[ : , "Region"][ final_df.loc[ : , 'Cluster' ] == '3']

         .value_counts(normalize = True)
         .round(decimals = 2)
         .sort_index().to_string(dtype = False, name = False)}
""")

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

~~~

                               _             _ 
     /\                       (_)           | |
    /  \   _ __ ___   __ _ _____ _ __   __ _| |
   / /\ \ | '_ ` _ \ / _` |_  / | '_ \ / _` | |
  / ____ \| | | | | | (_| |/ /| | | | | (_| |_|
 /_/    \_\_| |_| |_|\__,_/___|_|_| |_|\__, (_)
                                        __/ |  
                                       |___/                                                             
~~~

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<br>